In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import jieba
import re
import os
import time
import json
from datetime import datetime
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import joblib
from collections import Counter

In [4]:
# 设置随机种子，确保结果可重现
np.random.seed(42)

# 设置绘图样式
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'Microsoft YaHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号

In [7]:
data_dir = "data/THUCNews"

files = {
    'train': os.path.join(data_dir, 'cnews.train.txt'),
    'val': os.path.join(data_dir, 'cnews.val.txt'),
    'test': os.path.join(data_dir, 'cnews.test.txt'),
    'vocab': os.path.join(data_dir, 'cnews.vocab.txt')
}

for name, path in files.items():
    if not os.path.exists(path):
        print(f"❌ 文件不存在: {path}")
        if name == 'vocab':
            print("💡 词汇表文件不是必需的，可以继续，但建议检查数据完整性")
        else:
            raise FileNotFoundError(f"必需的数据文件不存在: {path}")


files

{'train': 'data/THUCNews/cnews.train.txt',
 'val': 'data/THUCNews/cnews.val.txt',
 'test': 'data/THUCNews/cnews.test.txt',
 'vocab': 'data/THUCNews/cnews.vocab.txt'}

In [8]:
# 读取数据
def read_news_file(file_path):
    """读取新闻文件，返回(texts, labels)"""
    texts = []
    labels = []

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                
                # 分割类别和内容(使用制表符分割)
                parts = line.split('\t')
                if len(parts) >= 2:
                    label = parts[0].strip()
                    content = parts[1].strip()
                    # 只保留足够长度的文本
                    if len(content) > 10:
                        texts.append(content)
                        labels.append(label)
        
        print(f"✅ 从 {file_path} 读取了 {len(texts)} 条新闻")
        return texts, labels
    
    except Exception as e:
        print(f"❌ 读取文件 {file_path} 时出错: {e}")
        return [], []


In [39]:
# 读取训练集
train_texts, train_labels = read_news_file(files['train'])

# 读取验证集
val_texts, val_labels = read_news_file(files['val'])

# 读取测试集
test_texts, test_labels = read_news_file(files['test'])

# 合并所有数据用于小样本训练
all_texts = train_texts + val_texts + test_texts
all_labels = train_labels + val_labels + test_labels

print(f"\n📊 数据统计信息:")
print(f"总样本数: {len(all_texts)}")
print(f"训练集: {len(train_texts)}, 验证集: {len(val_texts)}, 测试集: {len(test_texts)}")

label_counter = Counter(all_labels)
print("\n类别分布:")
for category, count in sorted(label_counter.items()):
    print(f"  {category}: {count} 个样本")


✅ 从 data/THUCNews/cnews.train.txt 读取了 49999 条新闻
✅ 从 data/THUCNews/cnews.val.txt 读取了 5000 条新闻
✅ 从 data/THUCNews/cnews.test.txt 读取了 10000 条新闻

📊 数据统计信息:
总样本数: 64999
训练集: 49999, 验证集: 5000, 测试集: 10000

类别分布:
  体育: 6500 个样本
  娱乐: 6500 个样本
  家居: 6500 个样本
  房产: 6499 个样本
  教育: 6500 个样本
  时尚: 6500 个样本
  时政: 6500 个样本
  游戏: 6500 个样本
  科技: 6500 个样本
  财经: 6500 个样本


In [12]:
categories = sorted(set(all_labels))
print(f"所有类别 ({len(categories)}类):")
print(categories)

所有类别 (10类):
['体育', '娱乐', '家居', '房产', '教育', '时尚', '时政', '游戏', '科技', '财经']


In [13]:
# 加载词汇表
vocab = None
try:
    with open(files['vocab'], 'r', encoding='utf-8') as f:
        vocab = [line.strip() for line in f if line.strip()]
        print(f"\n📚 词汇表信息:")
        print(f"词汇表大小: {len(vocab)} 个词")
        print(f"前10个词汇: {vocab[:10]}")
        print(f"特殊标记: {vocab[0]} (通常表示填充)")
except Exception as e:
    print(f"⚠️  加载词汇表时出错: {e}")



📚 词汇表信息:
词汇表大小: 4997 个词
前10个词汇: ['<PAD>', '，', '的', '。', '一', '是', '在', '0', '有', '不']
特殊标记: <PAD> (通常表示填充)


In [ ]:
# 加载中文停用词
stopwords_file = "data/stopwords_cn.txt"
stopwords = []

try:
    with open(stopwords_file, 'r', encoding='utf-8') as f:
        file_stopwords = [line.strip() for line in f if line.strip()]
        # 逐个添加
        stopwords.extend(file_stopwords)
        # 去重
        stopwords = set(stopwords)
        print(f"✅ 从文件加载了 {len(file_stopwords)} 个停用词")
except Exception as e:
    print(f"⚠️  加载停用词时出错: {e}")

stopwords

✅ 从文件加载了 849 个停用词


{'--',
 '<PAD>',
 '?',
 '“',
 '”',
 '》',
 '一',
 '一下',
 '一些',
 '一切',
 '一则',
 '一天',
 '一定',
 '一方面',
 '一旦',
 '一时',
 '一来',
 '一样',
 '一次',
 '一片',
 '一直',
 '一致',
 '一般',
 '一起',
 '一边',
 '一面',
 '万一',
 '上下',
 '上升',
 '上去',
 '上来',
 '上述',
 '上面',
 '下列',
 '下去',
 '下来',
 '下面',
 '不一',
 '不久',
 '不仅',
 '不会',
 '不但',
 '不光',
 '不单',
 '不变',
 '不只',
 '不可',
 '不同',
 '不够',
 '不如',
 '不得',
 '不怕',
 '不惟',
 '不成',
 '不拘',
 '不敢',
 '不断',
 '不是',
 '不比',
 '不然',
 '不特',
 '不独',
 '不管',
 '不能',
 '不要',
 '不论',
 '不足',
 '不过',
 '不问',
 '与',
 '与其',
 '与否',
 '与此同时',
 '专门',
 '且',
 '两者',
 '严格',
 '严重',
 '个',
 '个人',
 '个别',
 '中小',
 '中间',
 '丰富',
 '临',
 '为',
 '为主',
 '为了',
 '为什么',
 '为什麽',
 '为何',
 '为着',
 '主张',
 '主要',
 '举行',
 '乃',
 '乃至',
 '么',
 '之',
 '之一',
 '之前',
 '之后',
 '之後',
 '之所以',
 '之类',
 '乌乎',
 '乎',
 '乘',
 '也',
 '也好',
 '也是',
 '也罢',
 '了',
 '了解',
 '争取',
 '于',
 '于是',
 '于是乎',
 '云云',
 '互相',
 '产生',
 '人们',
 '人家',
 '什么',
 '什么样',
 '什麽',
 '今后',
 '今天',
 '今年',
 '今後',
 '仍然',
 '从',
 '从事',
 '从而',
 '他',
 '他人',
 '他们',
 '他的',
 '代替',
 '以',
 '以上',
 '以下',
 '以为',
 '以便',
 '

In [23]:
text = train_texts[0]

text = re.sub(r'https?://\S+|www\.\S+', '', text)   # 移除URL
text = re.sub(r'\S*@\S*\s?', '', text)  # 移除电子邮件地址
text = re.sub(r'[^\u4e00-\u9fa5]', '', text)    # 只保留汉字
text = re.sub(r'\s+', ' ', text).strip()    # 合并空白字符并去除首尾空格

# 2. 中文分词
if True:
    words = jieba.lcut(text)

# 4. 文本清洗
filtered_words = [
    word.strip() for word in words
    if word.strip() not in stopwords    # 不在停用词中
    and len(word.strip()) > 1       # 非单字词
    and not word.strip().isdigit()      # 不是纯数字
]

Processed_text = " ".join(filtered_words)


Processed_text[:50]

'马晓旭 意外 受伤 国奥 警惕 无奈 大雨 格外 青睐 殷家 记者 傅亚雨 沈阳 报道 来到 沈阳 '

In [24]:
# 中文预处理
def preprocess_cn_text(text, use_jieba=True):
    """
    Parameters:
    - use_jieba: 是否使用jieba分词
    """
    # 1. 移除特殊字符和噪声
    text = re.sub(r'https?://\S+|www\.\S+', '', text)   # 移除URL
    text = re.sub(r'\S*@\S*\s?', '', text)  # 移除电子邮件地址
    text = re.sub(r'[^\u4e00-\u9fa5]', '', text)    # 只保留汉字
    text = re.sub(r'\s+', ' ', text).strip()    # 合并空白字符并去除首尾空格

    if not text or len(text) < 5:
        return ""

    # 2. 中文分词
    if use_jieba:
        words = jieba.lcut(text)
    else:
        print("⚠️   简单按字符分割（不推荐，仅用于调试）")
        words = [char for char in text if char.strip()]

    # 3. 加载中文停用词
    # stopwords = stopwords
    
    # 4. 文本清洗
    filtered_words = [
        word.strip() for word in words
        if word.strip() not in stopwords    # 不在停用词中
        and len(word.strip()) > 1       # 非单字词
        and not word.strip().isdigit()      # 不是纯数字
    ]

    Processed_text = " ".join(filtered_words)

    return Processed_text


In [ ]:
# 批次处理
def batch_preprocess_texts(texts, batch_size=1000):
    """
    批量预处理文本，显示进度
    """
    print("\n🔄 批量预处理中文文本...")
    print(f"总样本数: {len(texts)}")
    
    processed_texts = []
    start_time = time.time()
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        batch_processed = [preprocess_cn_text(text) for text in batch]
        processed_texts.extend(batch_processed)
        
        # 显示进度
        progress = min(i + batch_size, len(texts))
        elapsed_time = time.time() - start_time
        avg_time_per_sample = elapsed_time / (i + batch_size)
        remaining_time = avg_time_per_sample * (len(texts) - progress)
        
        print(f"  进度: {progress}/{len(texts)} ({progress/len(texts)*100:.1f}%) "
              f"- 预计剩余时间: {remaining_time:.1f}秒")
    
    total_time = time.time() - start_time
    print(f"✅ 文本预处理完成！总耗时: {total_time:.2f}秒")

    return processed_texts

In [ ]:
# sample_size = 1000

# # 将标签转为数值
# label_encoder = LabelEncoder()
# y_encoder = label_encoder.fit_transform(categories)

# # 创建类别映射
# class_mapping = {i: category for i, category in enumerate(label_encoder.classes_)}
# print(f"类别映射: {class_mapping}")
# print(f"类别数量: {len(class_mapping)}")

# # 保证类别平衡的小样本选择
# # 按类别分组
# category_indices = {}
# for idx, label in enumerate(y_encoder):
#     if label not in category_indices:
#         category_indices[label] = []
#     category_indices[label].append(idx)

# # 为每个类别选择样本
# selected_indices = []
# samples_per_category = max(1, 1000 // len(class_mapping))

# for category, indices in category_indices.items():
#     # 随机选择该类别的样本
#     category_samples = min(len(indices), samples_per_category)
#     selected = np.random.choice(indices, category_samples, replace=False)
#     selected_indices.extend(selected)


# category_indices

类别映射: {0: '体育', 1: '娱乐', 2: '家居', 3: '房产', 4: '教育', 5: '时尚', 6: '时政', 7: '游戏', 8: '科技', 9: '财经'}
类别数量: 10


{0: [0],
 1: [1],
 2: [2],
 3: [3],
 4: [4],
 5: [5],
 6: [6],
 7: [7],
 8: [8],
 9: [9]}

In [ ]:
# def prepare_thucnews_small_sample(texts, labels, sample_size=1000, 
#                                  feature_type='tfidf', max_features=2000, 
#                                  use_pca=False, n_components=150):
#     """
#     准备THUCNews小样本数据集（适配标准格式）
    
#     Parameters:
#     - sample_size: 小样本大小
#     - feature_type: 特征类型 ('tfidf', 'count', 'binary')
#     - max_features: 最大特征数
#     - use_pca: 是否使用PCA降维
#     - n_components: PCA降维后的维度
#     """

#     print(f"🔧 准备THUCNews小样本数据集 (样本数: {sample_size})...")
    
#     # 转换标签为数值
#     label_encoder = LabelEncoder()
#     y_encoded = label_encoder.fit_transform(labels)
    
#     # 创建类别映射
#     class_mapping = {i: category for i, category in enumerate(label_encoder.classes_)}
#     print(f"类别映射: {class_mapping}")
#     print(f"类别数量: {len(class_mapping)}")
    
#     # 保证类别平衡的小样本选择
#     print("\n🔄 选择类别平衡的小样本...")
    
#     # 按类别分组
#     category_indices = {}
#     for idx, label in enumerate(y_encoded):
#         if label not in category_indices:
#             category_indices[label] = []
#         category_indices[label].append(idx)
    
#     # 为每个类别选择样本
#     selected_indices = []
#     samples_per_category = max(1, sample_size // len(class_mapping))
    
#     for category, indices in category_indices.items():
#         # 随机选择该类别的样本
#         category_samples = min(len(indices), samples_per_category)
#         selected = np.random.choice(indices, category_samples, replace=False)
#         selected_indices.extend(selected)
    
    

In [ ]:
train_labels_np = np.array(train_labels)
indices_tmp = np.where(train_labels_np == '房产')[0].tolist()

train_texts_category = train_texts[int(indices_tmp[0]): int(indices_tmp[-1]+1)]
len(indices_tmp),len(train_texts_category)

(4999, 4999)

In [ ]:
def create_thucnews_sample(texts, labels, categories, sample_size=1000,
                           feature_type='tfidf', max_features=2000, 
                           use_pca=False, n_components=150):
    """
    准备THUCNews小样本数据集（适配标准格式）
    
    Parameters:
    - sample_size: 小样本大小
    - feature_type: 特征类型 ('tfidf', 'count', 'binary')
    - max_features: 最大特征数
    - use_pca: 是否使用PCA降维
    - n_components: PCA降维后的维度
    """
    print(f"🔧 准备THUCNews小样本数据集 (样本数: {sample_size})...")

    # 检查样本数量
    if len(texts) < sample_size:
        print(f"⚠️  可用样本数({len(texts)})小于请求的样本数({sample_size})")
        sample_size = len(texts)
    
    # 1. 平衡采样
    label_array = np.array(labels)    # 转换成np,方便后续的处理
    samples_per_class = sample_size // len(categories)
    print(f"每类样本数: {samples_per_class}")

    selected_texts = []
    selected_labels = []

    for category in categories:
        # 获取该类别的所有索引
        indices = np.where(label_array == category)[0]
        
        if len(indices) == 0:
            print(f"⚠️  类别 '{category}' 没有样本")
            continue
        
        # 随机选取样本
        if len(indices) > samples_per_class:
            selected_indices = np.random.choice(indices, samples_per_class, replace=False)
        else:
            selected_indices = indices
        
        for idx in selected_indices:
            selected_texts.append(texts[idx])
            selected_labels.append(labels[idx])
    
    print(f"平衡采样后样本数: {len(selected_texts)}")

    # 2. 标签编码
    print("\n🏷️  编码标签...")
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(selected_labels)
    class_mapping = {i: class_name for i, class_name in enumerate(label_encoder.classes_)}
    print(f"类别映射: {class_mapping}")

    # 3. 文本预处理
    X_processed = batch_preprocess_texts(selected_texts)

    # 4. 特征提取
    print(f"\n📊 进行文本特征提取 ({feature_type.upper()})...")
    if feature_type == 'tfidf':
        # (?u)：Unicode模式
        # \b：单词边界
        # [\u4e00-\u9fa5]：匹配所有中文字符
        # +：一个或多个
        vectorizer = TfidfVectorizer(
            max_features=max_features,
            lowercase=False,    # 中文不需要大小写
            min_df=3,       # 增加最小文档频率，过滤罕见词
            max_df=0.9,     # 降低最大文档频率
            token_pattern=r'(?u)\b[\u4e00-\u9fa5]+\b',
            use_idf=True
        )
    elif feature_type == 'count':
        vectorizer = CountVectorizer(
            max_features=max_features,
            lowercase=False,
            min_df=3
        )
    
    # 拟合
    X_features = vectorizer.fit_transform(X_processed)
    feature_names = vectorizer.get_feature_names_out()

    print(f"特征提取后形状: {X_features.shape}")
    print(f"词汇表大小: {len(feature_names)}")
    
    # 5. 稠密矩阵
    print("\n🔄 转换为稠密矩阵...")
    X_dense = X_features.toarray()
    print(f"稠密矩阵形状: {X_dense.shape}")

    # 6. 数据标准化
    print("\n📊 数据标准化...")
    scaler = StandardScaler(with_mean=False)  # 稀疏数据使用with_mean=False
    X_scaled = scaler.fit_transform(X_dense)
    print(f"标准化后形状: {X_scaled.shape}")

In [ ]:
label_cnt = len(np.unique(train_labels_np))
label_cnt

10